# 🌧️ Rain Attenuation Calculator Notebook

This Jupyter notebook implements the ITU-R P.618 rain attenuation model for satellite links (1-4 GHz). Developed for Homework Problem 4: Follows Figure 2.4 flowchart, user inputs like Figure 2.5, and graphs results for Table 2.5 locations.

## Quick Start
1. **Install Dependencies**: `pip install matplotlib ipywidgets`  
2. **Run**: Open in Jupyter Notebook/Lab, execute all cells (Run All or Shift+Enter).  
3. **Interact**: Use widgets at bottom (location, f=1-4 GHz, pol=h/v, R≥0 mm/h, ε₀=1-90°). Click "Calculate" for results and bar graph (selected location highlighted).

## Features
- ✅ ITU-R P.618 steps: Interpolates Table 2.1 coeffs; computes γ, h_r, l_r, s, A_R.  
- ✅ Input validation; test case matches Figure 2.5 (Vienna).  
- ✅ Bar chart compares A_R across all locations.

## Model Parameters
| Parameter | Description | Range | Units |
|-----------|-------------|-------|-------|
| f | Frequency | 1.0-4.0 | GHz |
| pol | Polarization | h, v | - |
| R | Rain rate (0.01% exceedance) | ≥0 | mm/h |
| φ | Latitude | Per location | ° |
| hs | Height | ≥0 | km |
| ε₀ | Elevation | 1-90 | ° |

## Predefined Locations (Table 2.5)
| City | φ | hs (m) | City | φ | hs (m) |
|------|---|--------|------|---|--------|
| Madrid | 40.4 | 588 | Vienna | 48.2 | 193 |
| Tirana | 41.3 | 104 | Paris | 48.8 | 34 |
| Rome | 41.9 | 14 | Brussels | 50.8 | 76 |
| Pristina | 42.6 | 652 | London | 51.5 | 14 |
| Zagreb | 45.8 | 130 | Berlin | 52.5 | 34 |

## Example (Vienna: f=2, pol=v, R=30, ε₀=30 – matches Fig. 2.5)
- γ: 0.00252 dB/km  
- s: 0.8826  
- A_R: 0.0132 dB  

## Notes
- Valid for 1-4 GHz; low ε₀ may be unrealistic.  
- Edit cells for custom analysis.

## Honorable Mention
Extended to a Streamlit web app with maps/visuals: [https://rainattenuation.streamlit.app/](https://rainattenuation.streamlit.app/) (same core logic).

## References
- ITU-R P.618  
- Tables 2.1 & 2.5 from book excerpts (Ground Station Design for LEO Satellites, Ch. 1 & 3).

*Python 3, Matplotlib, ipywidgets • ITU-R Compliant • Submitted: September 28, 2025*

In [2]:
# Cell 1: Import all necessary libraries
import math  # For sin, exp, radians
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Constants for rain attenuation model
MIN_ELEVATION = 1.0  # Minimum elevation angle in degrees
EXP_COEFFICIENT = -0.015  # Exponential coefficient for reduction factor
REDUCTION_DENOMINATOR = 35  # Denominator constant for reduction factor
LAT_THRESHOLD = 23  # Latitude threshold for rain height calculation
RAIN_HEIGHT_BASE = 5  # Base rain height in km
RAIN_HEIGHT_COEFF = 0.073  # Coefficient for rain height adjustment

print("All libraries and constants loaded successfully!")

All libraries and constants loaded successfully!


In [3]:
# Cell 2: Table 2.1 - a and b coefficients for rain attenuation model
# Vertical polarization (v) - from table 2.1
coeff_v = {
    1.0: {'a': 0.0000308, 'b': .8592},
    1.5: {'a': 0.0000574, 'b': .8957},
    2.0: {'a': 0.0000998, 'b': .9490},
    2.5: {'a': 0.0001464, 'b': 1.0085},
    3.0: {'a': 0.0001942, 'b': 1.0688},
    3.5: {'a': 0.0002346, 'b': 1.1387},
    4.0: {'a': 0.0002461, 'b': 1.2476}
}

# Horizontal polarization (h) - from table 2.1
coeff_h = {
    1.0: {'a': 0.0000259, 'b': 0.9691},
    1.5: {'a': 0.0000443, 'b': 1.0185},
    2.0: {'a': 0.0000847, 'b': 1.0664},
    2.5: {'a': 0.0001321, 'b': 1.1209},
    3.0: {'a': 0.0001390, 'b': 1.2322},
    3.5: {'a': 0.0001155, 'b': 1.4189},
    4.0: {'a': 0.0001071, 'b': 1.6009}
}

print("Coefficient tables loaded:")
print("Vertical keys:", list(coeff_v.keys()))
print("Horizontal keys:", list(coeff_h.keys()))

Coefficient tables loaded:
Vertical keys: [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
Horizontal keys: [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]


In [4]:
# Cell 3: Interpolation function for frequencies not in table 2.1

def interpolate_coeff(f, coeff_dict):
    """
    Linearly interpolate a and b coefficients for frequency f (GHz) 
    between nearest values 
    """
    frequencies = sorted(coeff_dict.keys())
    
    if f < frequencies[0] or f > frequencies[-1]:
        raise ValueError(f"Frequency {f} GHz is out of range [{frequencies[0]}-{frequencies[-1]}]")

    if f in coeff_dict:
        return coeff_dict[f]['a'], coeff_dict[f]['b'] 

    # Find the two frequencies that bracket f
    for i in range(len(frequencies) - 1):
        if frequencies[i] <= f <= frequencies[i + 1]:
            low_f, high_f = frequencies[i], frequencies[i + 1]
            low_a, low_b = coeff_dict[low_f]['a'], coeff_dict[low_f]['b']
            high_a, high_b = coeff_dict[high_f]['a'], coeff_dict[high_f]['b']    
            
            # Linear interpolation fraction
            frac = (f - low_f) / (high_f - low_f)
            
            # Interpolate a and b
            a = low_a + (high_a - low_a) * frac
            b = low_b + (high_b - low_b) * frac
            
            return a, b
    
    # Fallback (shouldn't reach here)
    return coeff_dict[f]['a'], coeff_dict[f]['b']

# Test the interpolation function
test_f = 2.25  # Between 2.0 and 2.5
a_v, b_v = interpolate_coeff(test_f, coeff_v)
print(f"Interpolated for f={test_f} GHz (vertical): a={a_v:.6f}, b={b_v:.4f}")

Interpolated for f=2.25 GHz (vertical): a=0.000123, b=0.9788


In [5]:
# Cell 4: locations dictionary from Table 2.5 - European city locations with latitude and altitude
locations = {
    'Madrid': {'phi': 40.4, 'hs_m': 588},
    'Tirana': {'phi': 41.3, 'hs_m': 104},
    'Rome': {'phi': 41.9, 'hs_m': 14},
    'Pristina': {'phi': 42.6, 'hs_m': 652},
    'Zagreb': {'phi': 45.8, 'hs_m': 130},
    'Vienna': {'phi': 48.2, 'hs_m': 193},
    'Paris': {'phi': 48.8, 'hs_m': 34},
    'Brussels': {'phi': 50.8, 'hs_m': 76},
    'London': {'phi': 51.5, 'hs_m': 14},
    'Berlin': {'phi': 52.5, 'hs_m': 34}
}

print("Locations loaded:")
for city, data in locations.items():
    print(f"{city}: φ={data['phi']}°, h_s={data['hs_m']} m")

Locations loaded:
Madrid: φ=40.4°, h_s=588 m
Tirana: φ=41.3°, h_s=104 m
Rome: φ=41.9°, h_s=14 m
Pristina: φ=42.6°, h_s=652 m
Zagreb: φ=45.8°, h_s=130 m
Vienna: φ=48.2°, h_s=193 m
Paris: φ=48.8°, h_s=34 m
Brussels: φ=50.8°, h_s=76 m
London: φ=51.5°, h_s=14 m
Berlin: φ=52.5°, h_s=34 m


In [6]:
# Cell 5: Core function to calculate rain attenuation (follows Figure 2.4 flowchart)
def calculate_rain_attenuation(f, pol, R, phi, hs_km, epsilon0):
    """Calculate rain attenuation using ITU-R P.618 model.

    Args:
        f: Frequency in GHz
        pol: Polarization ('h' or 'v')
        R: Rain rate in mm/h
        phi: Latitude in degrees
        hs_km: Station height in km
        epsilon0: Elevation angle in degrees

    Returns:
        Dict with gamma, h_r, l_r, s, and A_R values
    """
    # Input validation
    if not (1.0 <= f <= 4.0):
        raise ValueError(f"Frequency {f} GHz out of range [1.0-4.0]")
    if pol not in ['h', 'v']:
        raise ValueError("Polarization must be 'h' or 'v'")
    if R < 0:
        raise ValueError("Rain rate must be non-negative")
    if not (MIN_ELEVATION <= epsilon0 <= 90):
        raise ValueError(f"Elevation angle must be between {MIN_ELEVATION}° and 90°")

    # Step 1: Get a and b using interpolation
    coeff_dict = coeff_v if pol == 'v' else coeff_h
    a, b = interpolate_coeff(f, coeff_dict)

    # Step 2: Calculate specific attenuation gamma
    gamma = a * (R ** b)

    # Step 3: Decision for h_r (effective rain height in km)
    if phi > LAT_THRESHOLD:
        h_r = RAIN_HEIGHT_BASE - RAIN_HEIGHT_COEFF * (phi - LAT_THRESHOLD)
    else:
        h_r = RAIN_HEIGHT_BASE

    # Step 4: Calculate slant path length l_r
    epsilon0_rad = math.radians(epsilon0)
    sin_epsilon = math.sin(epsilon0_rad)
    l_r = (h_r - hs_km) / sin_epsilon

    # Step 5: Calculate horizontal reduction factor s
    exp_term = math.exp(EXP_COEFFICIENT * R)
    denominator = REDUCTION_DENOMINATOR * exp_term
    s = 1 / (1 + (l_r * sin_epsilon) / denominator)

    # Step 6: Calculate total attenuation A_R
    A_R = gamma * s * l_r

    return {
        'gamma': gamma,
        'h_r': h_r,
        'l_r': l_r,
        's': s,
        'A_R': A_R
    }

# Test with Vienna example from Figure 2.5 (f=2, pol='v', R=30, phi=48.2, hs_m=193, epsilon0=30)
hs_km = 193 / 1000  # Convert m to km
results = calculate_rain_attenuation(2, 'v', 30, 48.2, hs_km, 30)
print("Test results for Vienna:")
print(results)

Test results for Vienna:
{'gamma': 0.002517205152366363, 'h_r': 3.1604, 'l_r': 5.934800000000001, 's': 0.8826390294247574, 'A_R': 0.013185840790267767}


In [7]:
# Cell 6: Utility function for location data extraction
def get_location_data(location):
    """Extract latitude and height data for a given location."""
    if location not in locations:
        raise ValueError(f"Invalid location: {location}. Available: {list(locations.keys())}")
    return locations[location]['phi'], locations[location]['hs_m'] / 1000

print("Utility functions loaded successfully!")

Utility functions loaded successfully!


In [8]:
# Cell 7: Test calculation with Vienna example from Figure 2.5
# (f=2, pol='v', R=30, phi=48.2, hs_m=193, epsilon0=30)
phi, hs_km = get_location_data('Vienna')
results = calculate_rain_attenuation(2, 'v', 30, phi, hs_km, 30)
print("Test results for Vienna (matches Figure 2.5):")
print(f"Specific Attenuation: {results['gamma']:.6f} dB/km")
print(f"Reduction Factor: {results['s']:.4f}")
print(f"Total Attenuation: {results['A_R']:.6f} dB")

Test results for Vienna (matches Figure 2.5):
Specific Attenuation: 0.002517 dB/km
Reduction Factor: 0.8826
Total Attenuation: 0.013186 dB


In [ ]:
# Cell 8: Interactive UI with widgets (like Figure 2.5 calculator)
# Output area for results
output = widgets.Output()

# Widgets for inputs (with defaults from problems)
location_dropdown = widgets.Dropdown(
    options=list(locations.keys()),
    value='Madrid',
    description='Location:'
)

f_slider = widgets.FloatSlider(
    value=2.5,
    min=1.0,
    max=4.0,
    step=0.1,
    description='Frequency (GHz):'
)

pol_radio = widgets.RadioButtons(
    options=['h', 'v'],
    value='v',
    description='Polarization:'
)

r_slider = widgets.FloatSlider(
    value=30,
    min=0,
    max=100,
    step=1,
    description='Rain Rate (mm/h):'
)

epsilon0_slider = widgets.FloatSlider(
    value=15,
    min=1,
    max=90,
    step=1,
    description='Elevation (°):'
)

calculate_button = widgets.Button(description="Calculate")

# Function to handle button click
def on_calculate_clicked(b):
    with output:
        clear_output(wait=True)
        try:
            # Get values
            location = location_dropdown.value
            f = f_slider.value
            pol = pol_radio.value
            R = r_slider.value
            epsilon0 = epsilon0_slider.value

            # Get phi and hs_km using utility function
            phi, hs_km = get_location_data(location)

            # Compute single location
            results = calculate_rain_attenuation(f, pol, R, phi, hs_km, epsilon0)

            # Display results (like Figure 2.5)
            print(f"Results for {location} (f={f} GHz, pol={pol}, R={R} mm/h, ε0={epsilon0}°):") 
            print(f"Specific Attenuation: {results['gamma']:.5f} dB/km")
            print(f"Reduction Factor: {results['s']:.4f}")
            print(f"Attenuation: {results['A_R']:.4f} dB")

            # Calculate for all locations for comparison plot
            cities = []
            a_r_values = []
            for city in locations.keys():
                phi_city, hs_km_city = get_location_data(city)
                results_city = calculate_rain_attenuation(f, pol, R, phi_city, hs_km_city, epsilon0)
                cities.append(city)
                a_r_values.append(results_city['A_R'])

            # Plot comparison
            plt.figure(figsize=(12, 6))
            bars = plt.bar(cities, a_r_values, color='skyblue', alpha=0.7)
            # Highlight selected location
            selected_idx = cities.index(location)
            bars[selected_idx].set_color('orange')
            bars[selected_idx].set_alpha(1.0)

            plt.xlabel('Location')
            plt.ylabel('Rain Attenuation A_R (dB)')
            plt.title(f'Rain Attenuation Across Locations\n(f={f} GHz, pol={pol}, R={R} mm/h, ε0={epsilon0}°)')
            plt.xticks(rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error: {e}")

# Link button to function
calculate_button.on_click(on_calculate_clicked)

# Display UI
display(widgets.VBox([
    location_dropdown,
    f_slider,
    pol_radio,
    r_slider,
    epsilon0_slider,
    calculate_button,
    output
]))